# Tutorial 09 - Kick-Starting your Final Project

Reese Dominguez - 100775764


I will partially answer the question "**What proportion of Sunshine Listers are women?**" As we will analyze the dataset for many of the Sunshine Lists over the years, I have decided to just analyze the proportion of women in **2020's Sunshine List**.



In [48]:
import pandas as pd
import numpy as np

In [2]:
# Importing the required datasets needed to answer this question
sunshine_2020 = pd.read_csv("sunshine2020.csv")
name_genders = pd.read_csv("wgnd.csv")

### Analysis of the World Gender Names Dictionary

There are many first names in the WGND, and all names seem to be lowercased. The version of the WGND I am using (wgnd_2_0_name-gender-langcode.csv) has three fields:

- **Name**: a first name
- **Langcode**: the short-form code of the language that the name either is in or comes from
- **Gender**: the associated gender with this name in this language

In [3]:
name_genders

,name,langcode,gender
0,"""baby""",en,F
1,'aisyah,en,F
2,'anela,en,F
3,'anela,fr,F
4,'fiyinfoluwa,en,F
...,...,...,...
21831038,히카리,ko,F
21831039,히토미,ko,F
21831040,힘찬,ko,M
21831041,凉峰,zh,M


In [4]:
# Looking at Russian names, as Russian has an alphabet separate from the Latin one.
# This is to check if names in different languages are lowercased, if the language's
# alphabet/syllabary allows for lowercasing.
name_genders[name_genders['langcode'] == 'ru']

,name,langcode,gender
152335,abram,ru,M
189067,aculina,ru,F
189246,ada,ru,F
192154,adam,ru,M
210441,adela,ru,F
...,...,...,...
21172001,яна,ru,F
21172004,ярик,ru,M
21172006,ярослав,ru,M
21172009,ярослава,ru,F


In [5]:
# A name that has multiple genders, depending on the language code
name_genders[name_genders['name'] == 'aubrey']

,name,langcode,gender
1361932,aubrey,af,M
1361933,aubrey,ak,M
1361934,aubrey,ar,M
1361935,aubrey,bn,M
1361936,aubrey,ee,M
1361937,aubrey,el,M
1361938,aubrey,en,M
1361939,aubrey,ff,M
1361940,aubrey,fr,M
1361941,aubrey,gu,M


In [6]:
# How many entries of each name are there?
name_genders['name'].value_counts()

nina     71
olga     70
diana    68
rita     67
eva      67
         ..
坡燉        1
坡燕        1
坡申        1
坡石        1
凉翼        1
Name: name, Length: 3505318, dtype: int64

In [10]:
# Capitalizing names to prepare for merge with Sunshine List
name_genders['name'] = name_genders['name'].str.capitalize()

In [8]:
# changing gender to int form to find the mode/most common gender
# 0 for M, 1 for F
def change_gender(gender):
    return int(gender == 'F')

name_genders['gender'] = name_genders['gender'].apply(change_gender)

In [11]:
# checking if capitalize and gender change worked
name_genders.iloc[1337:1337+5]

,name,langcode,gender
1337,Aabhas,te,0
1338,Aabheda,gu,1
1339,Aabheda,hi,1
1340,Aabheda,kn,1
1341,Aabheda,mr,1


In [26]:
# Removing all non-Latin names from this dataset as the Sunshine List only has Latin names

name_genders['name'] = name_genders['name'].apply(str)

name_genders = name_genders[name_genders['name'].map(lambda x: x.isascii())]

In [42]:
# This takes a very long time.

wgnd_condensed = name_genders.groupby(['name'])['gender'].agg(pd.Series.mode).to_frame()

TypeError: 'function' object is not subscriptable

This next code snippet will reduce any arrays in the `gender` column to only one value, for ease of calculations. **This will decrease the accuracy of any data we get!**

In [50]:
def get_first(x):
    if type(x) is np.ndarray:
        return x.flatten()[0]
    else:
        return x

wgnd_condensed['gender'] = wgnd_condensed['gender'].apply(get_first)

### Cleanup of the Sunshine LIst

The Sunshine List has been analyzed during the proposal. I set up/clean up the data to prepare for aggregation with the WGND.

In [52]:
# Change Salary paid and Taxable benefits to be a more easier datatype to sort w/
sunshine_2020["Salary paid"] = sunshine_2020["Salary paid"].replace('[\$,]', '', regex=True).astype(float)
sunshine_2020["Taxable benefits"] = sunshine_2020["Taxable benefits"].replace('[\$,]', '', regex=True).astype(float)

In [53]:
sunshine_2020

,Sector,Last name,First name,Salary paid,Taxable benefits,Employer,Job title,Calendar year
0,Colleges,Aarts,Cheri,113586.57,71.01,Fanshawe College Of Applied Arts and Technology,Professor,2020
1,Colleges,Aaslepp,Drew,113685.85,117.26,Humber College Institute Of Technology and Adv...,Professor,2020
2,Colleges,Abba,Corinne,102214.17,117.36,George Brown College Of Applied Arts and Techn...,Librarian,2020
3,Colleges,Abbott,Brian,102855.46,117.36,Conestoga College Institute Of Technology and ...,Professor,2020
4,Colleges,Abbott,Kathleen,160854.89,326.33,George Brown College Of Applied Arts and Techn...,"Associate Dean, Centre for Continuous Learning",2020
...,...,...,...,...,...,...,...,...
205601,Universities,Zylberberg,Joel,141478.88,727.20,York University,Assistant Professor / Canada Research Chair,2020
205602,Universities,Zylla,Phil,127898.47,231.93,McMaster Divinity College,Vice President Academic,2020
205603,Universities,Zytaruk,Nicole,113582.77,231.93,McMaster University,Research Associate,2020
205604,Universities,Zytner,Richard,193168.37,1906.08,University Of Guelph,Professor,2020


In [54]:
# So I don't have to type the columns in the next cell

new_cols = list(sunshine_2020.columns)
new_cols.append('gender')

print(new_cols)

['Sector', 'Last name', 'First name', 'Salary paid', 'Taxable benefits', 'Employer', 'Job title', 'Calendar year', 'gender']


In [55]:
# Merging the WGND (cleaned up) and 2020's Sunshine List
sunshine_wgnd = pd.merge(sunshine_2020, wgnd_condensed, left_on=['First name'], right_on=['name'])[new_cols]

sunshine_wgnd

,Sector,Last name,First name,Salary paid,Taxable benefits,Employer,Job title,Calendar year,gender
0,Colleges,Aarts,Cheri,113586.57,71.01,Fanshawe College Of Applied Arts and Technology,Professor,2020,1
1,Crown Agencies,Mara,Cheri,294772.37,10566.38,Ontario Cannabis Retail Corporation,Chief Commercial Officer / Directrice générale...,2020,1
2,Government of Ontario — Ministries,Hayward,Cheri,167653.20,218.04,Education,"Director, School Board Business Support Branch...",2020,1
3,Government of Ontario — Ministries,Huffman,Cheri,117777.29,148.58,"Children, Community and Social Services",Business Manager and Comptroller / Chef de la ...,2020,1
4,Government of Ontario — Ministries,Rockefeller,Cheri,129912.05,153.72,Ontario Provincial Police,Law Enforcement Officer / Agente d’exécution d...,2020,1
...,...,...,...,...,...,...,...,...,...
184122,Universities,Zuberi,Daniyal,176340.96,358.74,University Of Toronto,"Professor of Social Work, Professor of School ...",2020,0
184123,Universities,Zuk,Ireneus,190458.36,292.14,Queen’s University,Professor,2020,0
184124,Universities,Zunic,Nikolaj,157838.40,409.04,St. Jerome’s University,Associate Professor,2020,0
184125,Universities,Zwarenstein,Merrick,193254.60,95.60,University Of Western Ontario,Professor,2020,0


Around 20,000 rows were dropped, meaning that there might not be data in the `wgnd_condensed` dataset that's in the `sunshine_2020` dataset. This doesn't seem that bad, but there may be a loss of accuracy.

Now to figure out how many rows have `gender = 1`.

In [57]:
sunshine_wgnd['gender'].value_counts(normalize=True) * 100

0    51.732771
1    48.267229
Name: gender, dtype: float64

It seems that 48.26% of employees in the Sunshine List (that can have an assignable gender based on their name) may be female. This has sparked more interest in this number in particular:

- How many of these women in the list are/may be not white, compared to those who are?
- What public sectors do they tend to work in?
- What's the average salary paid of women compared to men?